In [13]:
import pandas as pd
import requests
import datetime
from datetime import date
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# Data from https://optimistic.etherscan.io/

In [14]:
# Contract creator ids
creator_ids = [
    '0xdead1cb30B3CA13Cd67D1d6f4e2790d12484Fdd8',
    '0x03863f6ad36f1fcd908517e3c56c6b3fd3f50752'
]


In [19]:
# Get contracts
df = pd.DataFrame()
apid = []
for addr in creator_ids:
    api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address=' + addr + '&startblock=0&endblock=99999999&sort=asc&apikey=YourApiKeyToken'
    r = requests.get(api_str, headers=headers)
    x = r.json()
    x = pd.DataFrame(x)
    x = pd.json_normalize(x['result'])
    x = x.loc[(x['to'] == '') & (x['contractAddress'] != '')] #Where transactions are being sent to NULL (means contract creation)
    apid.append(x)

df = pd.concat(apid)
df = df.drop_duplicates() # just in case
contract_hashes = df['contractAddress'].to_numpy() #convert to an array
print(contract_hashes)

<Response [200]>
<Response [200]>
['0x350a791bfc2c21f9ed5d10980dad2e2638ffa7f6'
 '0x5fee5e73ddc6c0176acf542c225591c8d3e1ebc1'
 '0x971dc14055cbc313a536f9ffd048e07530bd75df'
 '0x2e4a187732166a0282e52527b931c96146ac7c58'
 '0x588e1f339910c21c7e4864048e37017aaff4cbc6'
 '0xad3dfa54004f0f5d296002bb091cac10eb8a4891'
 '0xc326371d4d866c6ff522e69298e36fe75797d358'
 '0x8ce8c13d816fe6daf12d6fd9e4952e1fc88850af'
 '0x74d6b50283ac1d651f9afdc33521e4c1e3332b78'
 '0x25e1c58040f27ecf20bbd4ca83a09290326896b3'
 '0xa969beb73d918f6100163cd0fba3c586c269bee1']


In [31]:
#Get txs
df = pd.DataFrame()
apid = []
for addr in contract_hashes:
    api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address=' + addr + '&startblock=5000&endblock=99999999&sort=desc&apikey=YourApiKeyToken'
    #about the start
    r = requests.get(api_str, headers=headers)
    x = r.json()
    x = pd.DataFrame(x)
    x = pd.json_normalize(x['result'])
    if not x.empty:
        x = x.loc[x['to'] == str.lower(addr)] #Where transactions are being sent to the addresses
    #print(x)
    apid.append(x)
df = pd.concat(apid)


In [29]:
df_test = df
df_test['timeStamp'] = pd.to_datetime(df_test['timeStamp'],unit='s').dt.floor('d') #time is in seconds - trunc to day
print(df_test.sort_values(by='timeStamp', ascending=False))

     blockNumber  timeStamp  \
1407     1257325 2021-09-06   
1408     1257335 2021-09-06   
1401     1242780 2021-09-06   
1402     1245262 2021-09-06   
1403     1246632 2021-09-06   
...          ...        ...   
7          87005 2021-07-21   
8          87007 2021-07-21   
9          87028 2021-07-21   
10         87034 2021-07-21   
1          85553 2021-07-21   

                                                   hash nonce  \
1407  0x392ff142ca131c5ad63dbad431141d5ac225a4b1bc3f...     1   
1408  0x0a2bf43cd8b25b751801f3f98275bf6c263014302b6d...     1   
1401  0x90304063bd51add6e85e46907d9088f1e94d810cd617...     8   
1402  0xb3e67a97efc3b02baca586c2a50cca4174fd471eb612...    15   
1403  0xa7ae3681a2e119cb9471b95778174a4a01b7858d7160...     5   
...                                                 ...   ...   
7     0x7b4f73f7ac31245fe2fd13f007fdc084e1d91c130f1e...     0   
8     0xa6a248604604f74ad16f4ff39eb7e268112e4afb0c07...     0   
9     0x1fc4d8a5bbe479e9bed3b86f68f6634485

In [30]:
#Aggregate values
df_trunc = pd.DataFrame()
df_trunc['dt'] = pd.to_datetime(df['timeStamp'],unit='s').dt.floor('d') #time is in seconds - trunc to day
df_trunc['hash'] = df['hash']
df_trunc['gas'] = pd.to_numeric(df['gas'])
df_trunc['gasPrice'] = pd.to_numeric(df['gasPrice'])
df_trunc['gasUsed'] = pd.to_numeric(df['gasUsed'])

print(df_trunc.dtypes)
df_trunc=df_trunc.groupby('dt').agg({'hash':'count','gas':'sum', 'gasPrice':'mean','gasUsed':'sum'})\
        .rename(columns={"hash":"num_txs","gas": "total_gas", "gasPrice":"avg_gas_price", "gasUsed":"total_gas_used"})

print(df_trunc)

dt          datetime64[ns]
hash                object
gas                  int64
gasPrice             int64
gasUsed              int64
dtype: object
            num_txs     total_gas  avg_gas_price  total_gas_used
dt                                                              
2021-07-21      638   13255282699       15000000       871446397
2021-07-22     1251   30591262499       15000000      1743325983
2021-07-23     5533  126296836548       15000000      7582254535
2021-07-24     2631   39203649225       15000000      3573193268
2021-07-25      723    8244144183       15000000       959947365
2021-07-26     4575  144853787781       15000000      6226241622
2021-07-27     9344  306921676420       15000000     12892129658
2021-07-28     8818  222722399529       15000000     12265973386
2021-07-29     6055  118644412025       15000000      8309116285
2021-07-30      699   18322084244       15000000       966785612
2021-07-31       17     303550272       15000000        13537813
2021-0

In [24]:
#df_export = pd.DataFrame()
#df_export['dt'] = pd.to_datetime(df['timeStamp'],unit='s') #time is in seconds
#df_export['hash'] = df['hash']
#df_export['to'] = df['to']
#df_export['gas'] = df['gas']
#df_export['gasPrice'] = df['gasPrice']
#df_export['gasUsed'] = df['gasUsed']

df_export = df_trunc
df_export = df_export.drop_duplicates() # just in case
export_str = 'optimistic_etherscan_oracle_' + date.today().strftime("%Y_%m_%d") + '.csv'

df_export.to_csv(export_str,index=True)